In [ ]:
!pip install ray==1.12.0

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/CS7643 Final Project/'  

In [ ]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total

In [ ]:
from data.get_dataset import GetDataset, StockData

FAKE_DATA=False
REPEAT_ONE_SMALL_BATCH=False
NUM_DAYS = 5
FUTURE_DAYS= 5
DROP_FIRST_N_DAYS=10000
#USE_TRANSFORMER=False #ONLY DOES FULLY CONNECTED FOR NOW
#USE_ALT_TRANSFORMER = False  #ONLY DOES FULLY CONNECTED FOR NOW
NUM_SAMPLES=10 # FOR RAY TUNE
MAX_NUM_EPOCHS=10
CONFIG = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "batch_size": tune.choice([32])
    #"batch_size": tune.choice([2, 4]),
    #"dim_feedforward": tune.sample_from(lambda _: 2 ** np.random.randint(4, 9)),#16 to 256
    #"num_layers": tune.choice([1, 2, 3]),
    #"dropout": tune.loguniform(1e-5, 5e-1)
}

csv = './data/SPXDailyData.csv'
df = GetDataset(csv)
dataset = df.get_data2(FUTURE_DAYS)
dataset = dataset[:][DROP_FIRST_N_DAYS:]
if REPEAT_ONE_SMALL_BATCH:
  dataset = dataset[0:10]
  if FAKE_DATA:
    dataset[:][0:5]=.6
    dataset[:][5:]=.4
    dataset['Next_Day_Change'][0:5]=0
    dataset['Next_Day_Change'][5:]=1

#split into 3
valid_frac, test_frac = 0.2, 0.2
train_sz=int(dataset.shape[0]*(1-(valid_frac+test_frac)))
valid_sz=int(dataset.shape[0]*(valid_frac))
df_train = dataset[               0:train_sz]
df_valid = dataset[        train_sz:train_sz+valid_sz]
df_test = dataset[train_sz+valid_sz:]

#convert to sequence data and make dataset
train_dataset = StockData(df_train.to_numpy(), num_days=NUM_DAYS)
valid_dataset = StockData(df_valid.to_numpy(), num_days=NUM_DAYS) 
test_dataset = StockData(df_test.to_numpy(), num_days=NUM_DAYS) 


In [ ]:
class FCNet(nn.Module):
  def __init__(self,in_shape):
    super(FCNet,self).__init__()
    self.fc_layer1 = nn.Linear(in_shape,in_shape*4)
    self.bn_layer1 = nn.BatchNorm1d(in_shape*4)
    self.fc_layer2 = nn.Linear(in_shape*4,in_shape*8)
    self.bn_layer2 = nn.BatchNorm1d(in_shape*8)
    self.fc_layer3 = nn.Linear(in_shape*8,1)
  def forward(self,x):
    x = torch.flatten(x,start_dim=1)
    x = self.bn_layer1(torch.relu(self.fc_layer1(x)))
    x = self.bn_layer2(torch.relu(self.fc_layer2(x)))
    x = self.fc_layer3(x)
    return x.reshape(x.shape[0],-1)
#class FCNet(nn.Module):
#  def __init__(self,in_shape):
#    super(FCNet,self).__init__()
#    self.fc_layer1 = nn.Linear(in_shape,in_shape//2)
#    self.bn_layer1 = nn.BatchNorm1d(in_shape//2)
#    self.fc_layer2 = nn.Linear(in_shape//2,1)
#  def forward(self,x):
#    x = torch.flatten(x,start_dim=1)
#    x = self.bn_layer1(torch.relu(self.fc_layer1(x)))
#    x = self.fc_layer2(x)
#    return x.reshape(x.shape[0],-1)

#class FCNet(nn.Module):
#  def __init__(self,in_shape):
#    super(FCNet,self).__init__()
#    self.fc_layer1 = nn.Linear(in_shape,1)
#  def forward(self,x):
#    x = torch.flatten(x,start_dim=1)
#    x = self.fc_layer1(x)
#    return x.reshape(x.shape[0],-1)    
#print(train_dataset[:][0].shape[2])

modelFC = FCNet((train_dataset[:][0].shape[2])*NUM_DAYS)

In [ ]:
from utils import plot_curves
from utils import train
from utils import evaluate
from sklearn.exceptions import UndefinedMetricWarning
import copy

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def train_self(config, checkpoint_dir=None, data_dir=None):
   device = "cpu"
   model = None
   if torch.cuda.is_available():
      device = "cuda:0"
      model=modelFC
      if torch.cuda.device_count() > 1:
         model = nn.DataParallel(model)
   else:
      model=modelFC
   model.to(device)

   criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
   optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

   if checkpoint_dir:
      model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
      model.load_state_dict(model_state)
      optimizer.load_state_dict(optimizer_state)
  
   train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8)
   valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8)
    

   avg_train_loss,avg_train_acc,avg_valid_loss,avg_valid_acc=[],[],[],[]
   model.float()
   for epoch in range(MAX_NUM_EPOCHS):
      train_loss, atl, ata = train(model, train_loader, optimizer, criterion, device)
      #scheduler.step(train_loss)
      _, avl, ava = evaluate(model, valid_loader, criterion, device)
      if epoch%50==1:
         print("Epoch %d: Training Loss: %.4f. Training Acc: %.4f. Validation Loss: %.4f. Validation Acc: %.4f." % (epoch+1, atl, ata, avl, ava))
      avg_train_loss.append(atl.item())
      avg_train_acc.append(ata)
      avg_valid_loss.append(avl.item())
      avg_valid_acc.append(ava)

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

      tune.report(loss=(avl.item()), accuracy=ava)
 

In [ ]:
#cite: https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
GPUS_PER_TRIAL=2

data_dir = os.path.abspath("./data")

config=CONFIG#see above
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=MAX_NUM_EPOCHS,
    grace_period=1,
    reduction_factor=2)
reporter = JupyterNotebookReporter(
    True, 
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])
#result = tune.run(
#    partial(train_self, data_dir=data_dir),
#    resources_per_trial={"cpu": 2, "gpu": GPUS_PER_TRIAL},
#    config=config,
#    num_samples=NUM_SAMPLES,
#    scheduler=scheduler,
#    progress_reporter=reporter)

result = tune.run(
    train_self, 
    config=config, 
    verbose=1,
    scheduler=scheduler,
    num_samples=NUM_SAMPLES,
    progress_reporter=reporter# This is used to specify the logging directory.   
    #stop={"loss": 0.3}  # This will stop the trial 
)

print('=========================================')
#print(result.results_df)
#print(result.trial_dataframes)
print('=========================================')
best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["accuracy"]))

ax = None  # This plots everything on the same plot
for d in result.trial_dataframes.values():
    ax = d.loss.plot(ax=ax, legend=False)
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
plt.show()

ax = None  # This plots everything on the same plot
for d in result.trial_dataframes.values():
    ax = d.accuracy.plot(ax=ax, legend=False)
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
plt.show()


best_trained_model = None
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if GPUS_PER_TRIAL > 1:
        best_trained_model = modelFC
        best_trained_model = nn.DataParallel(best_trained_model)
else:
   best_trained_model = modelFC
best_trained_model.to(device)

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)


#Todo later -> use eval
#test_acc = test_accuracy(best_trained_model, device)
#print("Best trial test set accuracy: {}".format(test_acc))